 importing all essential packages

In [0]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

In [0]:
data = pd.read_excel("data.xlsx")

In [4]:
data.describe(include="all")

,Disease,Alias,Overview,Symptoms,Causes,Risk factors,Complications,Prevention,Images
count,2202,1019,2156,2100,2091,1788,1648,1165,1310
unique,2199,648,1145,1104,1093,926,864,606,603
top,Lung collapse,Viral hemorrhagic fevers,"['Dengue', 'Ebola', 'Lassa', 'Marburg', 'Yello...","['High fever', 'Fatigue', 'Dizziness', 'Muscle...",The viruses that cause viral hemorrhagic fever...,"['Working with the sick', 'Slaughtering infect...","['Brain', 'Eyes', 'Heart', 'Kidneys', 'Liver',...",['Keep pet food covered and stored in rodent-p...,['https://www.mayoclinic.org/-/media/kcms/gbs/...
freq,2,8,9,9,9,9,9,9,20


 checking for null values

In [5]:
data.isnull().sum()

Disease             0
Alias            1183
Overview           46
Symptoms          102
Causes            111
Risk factors      414
Complications     554
Prevention       1037
Images            892
dtype: int64

In [0]:
data.drop("Images",axis = 1, inplace = True)

In [7]:
data.columns

Index(['Disease', 'Alias', 'Overview', 'Symptoms', 'Causes', 'Risk factors',
       'Complications', 'Prevention'],
      dtype='object')

In [8]:
data.head()

,Disease,Alias,Overview,Symptoms,Causes,Risk factors,Complications,Prevention
0,A fib,Atrial fibrillation,Atrial fibrillation is an irregular and often ...,"['Palpitations, which are sensations of a raci...","['High blood pressure', 'Heart attack', 'Coron...","['Age. The older you are, the greater your ris...","[""Stroke. In atrial fibrillation, the chaotic ...","['Eating a heart-healthy diet', 'Increasing yo..."
1,Abdominal aortic aneurysm,NaN,An abdominal aortic aneurysm is an enlarged ar...,"['Deep, constant pain in your abdomen or on th...",['Hardening of the arteries (atherosclerosis)....,['Tobacco use. Smoking is the strongest risk f...,"['Sudden, intense and persistent abdominal or ...","[""Don't use tobacco products. Quit smoking or ..."
2,Abnormally excessive sweating,Hyperhidrosis,Hyperhidrosis (hi-pur-hi-DROE-sis) is abnormal...,"['Sweating disrupts your daily routine', 'Swea...","['Diabetes', 'Menopause hot flashes', 'Thyroid...",NaN,['Infections. People who sweat profusely are m...,NaN
3,"Abscess, Bartholin's",Bartholin's cyst,The Bartholin's (BAHR-toe-linz) glands are loc...,"['A tender, painful lump near the vaginal open...",Experts believe that the cause of a Bartholin'...,NaN,A Bartholin's cyst or abscess may recur and ag...,There's no way to prevent a Bartholin's cyst. ...
4,Absence seizure,NaN,"Absence seizures involve brief, sudden lapses ...","['Sudden stop in motion without falling', 'Lip...",Many children appear to have a genetic predisp...,['Age. Absence seizures are more common in chi...,['Must take anti-seizure medications throughou...,NaN


In [0]:
target = data["Disease"]

In [10]:
target

0                               A fib 
1            Abdominal aortic aneurysm
2       Abnormally excessive sweating 
3                Abscess, Bartholin's 
4                      Absence seizure
                     ...              
2197                              Yips
2198                        Zika virus
2199        Zollinger-Ellison syndrome
2200                           Zoster 
2201        22q11.2 deletion syndrome 
Name: Disease, Length: 2202, dtype: object

In [0]:
data.drop("Disease",axis = 1, inplace = True)

In [12]:
data.head()

,Alias,Overview,Symptoms,Causes,Risk factors,Complications,Prevention
0,Atrial fibrillation,Atrial fibrillation is an irregular and often ...,"['Palpitations, which are sensations of a raci...","['High blood pressure', 'Heart attack', 'Coron...","['Age. The older you are, the greater your ris...","[""Stroke. In atrial fibrillation, the chaotic ...","['Eating a heart-healthy diet', 'Increasing yo..."
1,NaN,An abdominal aortic aneurysm is an enlarged ar...,"['Deep, constant pain in your abdomen or on th...",['Hardening of the arteries (atherosclerosis)....,['Tobacco use. Smoking is the strongest risk f...,"['Sudden, intense and persistent abdominal or ...","[""Don't use tobacco products. Quit smoking or ..."
2,Hyperhidrosis,Hyperhidrosis (hi-pur-hi-DROE-sis) is abnormal...,"['Sweating disrupts your daily routine', 'Swea...","['Diabetes', 'Menopause hot flashes', 'Thyroid...",NaN,['Infections. People who sweat profusely are m...,NaN
3,Bartholin's cyst,The Bartholin's (BAHR-toe-linz) glands are loc...,"['A tender, painful lump near the vaginal open...",Experts believe that the cause of a Bartholin'...,NaN,A Bartholin's cyst or abscess may recur and ag...,There's no way to prevent a Bartholin's cyst. ...
4,NaN,"Absence seizures involve brief, sudden lapses ...","['Sudden stop in motion without falling', 'Lip...",Many children appear to have a genetic predisp...,['Age. Absence seizures are more common in chi...,['Must take anti-seizure medications throughou...,NaN


In [0]:
data.fillna(" ",inplace = True)

In [14]:
data.head()

,Alias,Overview,Symptoms,Causes,Risk factors,Complications,Prevention
0,Atrial fibrillation,Atrial fibrillation is an irregular and often ...,"['Palpitations, which are sensations of a raci...","['High blood pressure', 'Heart attack', 'Coron...","['Age. The older you are, the greater your ris...","[""Stroke. In atrial fibrillation, the chaotic ...","['Eating a heart-healthy diet', 'Increasing yo..."
1,,An abdominal aortic aneurysm is an enlarged ar...,"['Deep, constant pain in your abdomen or on th...",['Hardening of the arteries (atherosclerosis)....,['Tobacco use. Smoking is the strongest risk f...,"['Sudden, intense and persistent abdominal or ...","[""Don't use tobacco products. Quit smoking or ..."
2,Hyperhidrosis,Hyperhidrosis (hi-pur-hi-DROE-sis) is abnormal...,"['Sweating disrupts your daily routine', 'Swea...","['Diabetes', 'Menopause hot flashes', 'Thyroid...",,['Infections. People who sweat profusely are m...,
3,Bartholin's cyst,The Bartholin's (BAHR-toe-linz) glands are loc...,"['A tender, painful lump near the vaginal open...",Experts believe that the cause of a Bartholin'...,,A Bartholin's cyst or abscess may recur and ag...,There's no way to prevent a Bartholin's cyst. ...
4,,"Absence seizures involve brief, sudden lapses ...","['Sudden stop in motion without falling', 'Lip...",Many children appear to have a genetic predisp...,['Age. Absence seizures are more common in chi...,['Must take anti-seizure medications throughou...,


combining all columns to make it one  

In [0]:
data["collective"] = data["Alias"]

In [0]:
for i in range(1,len(data.columns)):
    data["collective"]+=data[data.columns[i]]

In [17]:
data.isnull().any()

Alias            False
Overview         False
Symptoms         False
Causes           False
Risk factors     False
Complications    False
Prevention       False
collective       False
dtype: bool

In [0]:
sentences = data["collective"]

## Data Cleaning

Here i am removing all the special chracters only a-z or A-Z or 0-9 chracters will be in the data set 

In [1]:
def cleaning(sentences):
    words = []
    for s in sentences:
        clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
        w = word_tokenize(clean)
        #lemmatizing
        words.append([i.lower() for i in w])

    return words

In [2]:
cleaned_words=cleaning(sentences)

NameError: name 'sentences' is not defined

In [24]:
cleaned_words[0]

['atrial',
 'fibrillationatrial',
 'fibrillation',
 'is',
 'an',
 'irregular',
 'and',
 'often',
 'rapid',
 'heart',
 'rate',
 'that',
 'can',
 'increase',
 'your',
 'risk',
 'of',
 'strokes',
 'heart',
 'failure',
 'and',
 'other',
 'heart',
 'related',
 'complications',
 'during',
 'atrial',
 'fibrillation',
 'the',
 'heart',
 's',
 'two',
 'upper',
 'chambers',
 'the',
 'atria',
 'beat',
 'chaotically',
 'and',
 'irregularly',
 'out',
 'of',
 'coordination',
 'with',
 'the',
 'two',
 'lower',
 'chambers',
 'the',
 'ventricles',
 'of',
 'the',
 'heart',
 'atrial',
 'fibrillation',
 'symptoms',
 'often',
 'include',
 'heart',
 'palpitations',
 'shortness',
 'of',
 'breath',
 'and',
 'weakness',
 'episodes',
 'of',
 'atrial',
 'fibrillation',
 'may',
 'come',
 'and',
 'go',
 'or',
 'you',
 'may',
 'develop',
 'atrial',
 'fibrillation',
 'that',
 'doesn',
 't',
 'go',
 'away',
 'and',
 'may',
 'require',
 'treatment',
 'although',
 'atrial',
 'fibrillation',
 'itself',
 'usually',
 'isn

In [25]:
len(sentences)

2202

In [26]:
len(cleaned_words)

2202

coverting plain string into tokens 

In [0]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    token = Tokenizer(filters = filters)
    token.fit_on_texts(words)
    return token

returning the row with max length of the dataset 

In [0]:
def max_length(words):
    return(len(max(words, key = len)))

In [29]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))


Vocab Size = 16629 and Maximum length = 4670


encoding sentences with respect to tokenizer 

In [0]:
def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

In [0]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [32]:
encoded_doc[0]

[813,
 14207,
 839,
 12,
 23,
 508,
 3,
 54,
 400,
 30,
 684,
 11,
 9,
 96,
 7,
 26,
 2,
 2110,
 30,
 155,
 3,
 35,
 30,
 326,
 215,
 68,
 813,
 839,
 1,
 30,
 19,
 219,
 371,
 967,
 1,
 1730,
 2619,
 7138,
 3,
 5108,
 186,
 2,
 1304,
 13,
 1,
 219,
 249,
 967,
 1,
 1187,
 2,
 1,
 30,
 813,
 839,
 50,
 54,
 121,
 30,
 1780,
 373,
 2,
 279,
 3,
 241,
 738,
 2,
 813,
 839,
 15,
 1053,
 3,
 531,
 4,
 10,
 15,
 66,
 813,
 839,
 11,
 281,
 32,
 531,
 379,
 3,
 15,
 510,
 59,
 291,
 813,
 839,
 1317,
 63,
 255,
 32,
 120,
 353,
 25,
 12,
 6,
 174,
 178,
 53,
 11,
 119,
 765,
 582,
 59,
 6,
 736,
 1584,
 13,
 813,
 839,
 12,
 1,
 1915,
 5,
 66,
 17,
 576,
 282,
 1,
 371,
 967,
 2,
 1,
 30,
 71,
 17,
 576,
 1731,
 8,
 1,
 30,
 15,
 3039,
 5,
 35,
 447,
 3,
 91,
 5,
 968,
 17,
 205,
 1614,
 340,
 20,
 813,
 839,
 15,
 121,
 78,
 3,
 35,
 5109,
 5,
 549,
 5,
 3931,
 1,
 30,
 19,
 785,
 97,
 1780,
 45,
 18,
 1764,
 2,
 6,
 3292,
 1177,
 508,
 660,
 4,
 6,
 8968,
 7591,
 8,
 7,
 199,
 241,
 480,
 

it is to just coverting encoded sentences into square 2D matrix with the no: columns equal to max_length

In [0]:
def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [0]:
padded_doc = padding_doc(encoded_doc, max_length)

In [35]:
padded_doc[:5]

array([[  813, 14207,   839, ...,     0,     0,     0],
       [   23,   275,   521, ...,     0,     0,     0],
       [11598,  4135,  7141, ...,     0,     0,     0],
       [ 3221,    19, 11599, ...,     0,     0,     0],
       [ 1799,   301,   754, ...,     0,     0,     0]], dtype=int32)

In [36]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (2202, 4670)


cleaning the output 

In [0]:
cleaned_output = cleaning(target)

converting dataset in categories 

Before, data was not in a catergorical form nor in regression form, so i converted the diseases into  27 categories, diseases starting with A as 1 category, disesases starting with B as secong category and so on

In [0]:
joiner = "/n"

In [0]:
for i in range(len(cleaned_output)):
    cleaned_output[i] = joiner.join(cleaned_output[i])
    cleaned_output[i] = cleaned_output[i][0]

In [0]:
uniques_diseases = list(set(cleaned_output))

In [41]:
uniques_diseases

['p',
 'i',
 'w',
 'z',
 'b',
 'h',
 'x',
 'n',
 'd',
 'k',
 'j',
 'o',
 'f',
 'c',
 'q',
 'g',
 'u',
 'v',
 'e',
 'a',
 'm',
 'r',
 '2',
 'y',
 't',
 'l',
 's']

In [0]:
output_tokenizer = create_tokenizer(uniques_diseases, filters = '!"#$%&()*+,-:/;<=>?@[\]^`{|}~')

In [43]:
output_tokenizer.word_index

{'2': 23,
 'a': 20,
 'b': 5,
 'c': 14,
 'd': 9,
 'e': 19,
 'f': 13,
 'g': 16,
 'h': 6,
 'i': 2,
 'j': 11,
 'k': 10,
 'l': 26,
 'm': 21,
 'n': 8,
 'o': 12,
 'p': 1,
 'q': 15,
 'r': 22,
 's': 27,
 't': 25,
 'u': 17,
 'v': 18,
 'w': 3,
 'x': 7,
 'y': 24,
 'z': 4}

In [0]:
encoded_output = encoding_doc(output_tokenizer, cleaned_output)

In [45]:
encoded_output[:6]

[[20], [20], [20], [20], [20], [20]]

In [0]:
encoded_output = np.array(encoded_output)

In [47]:
encoded_output.shape

(2202, 1)

In [48]:
encoded_output

array([[20],
       [20],
       [20],
       ...,
       [ 4],
       [ 4],
       [23]])

using one hot encoder

In [0]:
def one_hot(encode):
    o = OneHotEncoder(sparse = False)
    return(o.fit_transform(encode))

In [50]:
encoded_output

array([[20],
       [20],
       [20],
       ...,
       [ 4],
       [ 4],
       [23]])

In [0]:
output_one_hot = one_hot(encoded_output)

In [52]:
output_one_hot.shape

(2202, 27)

## Defining Model

In [0]:
from sklearn.model_selection import train_test_split

shuffling data into test and train data  

In [0]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)

In [55]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (1761, 4670) and train_Y = (1761, 27)
Shape of val_X = (441, 4670) and val_Y = (441, 27)


defining model 

In [0]:
def create_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
    model.add(Bidirectional(LSTM(128)))
    #model.add(LSTM(128))
    model.add(Dense(32, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(27, activation = "softmax"))

    return model

In [57]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4670, 128)         2128512   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 27)                891       
Total params: 2,400,795
Trainable params: 272,283
Non-trainable params: 2,128,512
_________________________________________________________________


In [59]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 5, batch_size = 200, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 1761 samples, validate on 441 samples
Epoch 1/5
1761/1761 [==============================] - 487s 277ms/step - loss: 3.1754 - accuracy: 0.0948 - val_loss: 3.0770 - val_accuracy: 0.0998

Epoch 00001: val_loss improved from inf to 3.07697, saving model to model.h5
Epoch 2/5
1761/1761 [==============================] - 468s 266ms/step - loss: 3.1432 - accuracy: 0.0931 - val_loss: 3.0790 - val_accuracy: 0.1156

Epoch 00002: val_loss did not improve from 3.07697
Epoch 3/5
1761/1761 [==============================] - 461s 262ms/step - loss: 3.1269 - accuracy: 0.0977 - val_loss: 3.0392 - val_accuracy: 0.0998

Epoch 00003: val_loss improved from 3.07697 to 3.03917, saving model to model.h5
Epoch 4/5
1761/1761 [==============================] - 466s 265ms/step - loss: 3.1170 - accuracy: 0.0931 - val_loss: 3.0295 - val_accuracy: 0.1088

Epoch 00004: val_loss improved from 3.03917 to 3.02955, saving model to model.h5
Epoch 5/5
1761/1761 [==============================] - 476s 270ms/step 

In [0]:
model = load_model("model.h5")

In [0]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred

In [0]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("probabilty of disease %s  = %s" % (classes[i], (predictions[i])))

In [70]:
text = sentences[0]
text

'Atrial fibrillationAtrial fibrillation is an irregular and often rapid heart rate that can increase your risk of strokes, heart failure and other heart-related complications.During atrial fibrillation, the heart\'s two upper chambers (the atria) beat chaotically and irregularly — out of coordination with the two lower chambers (the ventricles) of the heart. Atrial fibrillation symptoms often include heart palpitations, shortness of breath and weakness.Episodes of atrial fibrillation may come and go, or you may develop atrial fibrillation that doesn\'t go away and may require treatment. Although atrial fibrillation itself usually isn\'t life-threatening, it is a serious medical condition that sometimes requires emergency treatment.A major concern with atrial fibrillation is the potential to develop blood clots within the upper chambers of the heart. These blood clots forming in the heart may circulate to other organs and lead to blocked blood flow (ischemia).Treatments for atrial fibri

testing model 

In [71]:
text = text
pred = predictions(text)
get_final_output(pred, uniques_diseases)

['atrial', 'fibrillationatrial', 'fibrillation', 'is', 'an', 'irregular', 'and', 'often', 'rapid', 'heart', 'rate', 'that', 'can', 'increase', 'your', 'risk', 'of', 'strokes', 'heart', 'failure', 'and', 'other', 'heart', 'related', 'complications', 'during', 'atrial', 'fibrillation', 'the', 'heart', 's', 'two', 'upper', 'chambers', 'the', 'atria', 'beat', 'chaotically', 'and', 'irregularly', 'out', 'of', 'coordination', 'with', 'the', 'two', 'lower', 'chambers', 'the', 'ventricles', 'of', 'the', 'heart', 'atrial', 'fibrillation', 'symptoms', 'often', 'include', 'heart', 'palpitations', 'shortness', 'of', 'breath', 'and', 'weakness', 'episodes', 'of', 'atrial', 'fibrillation', 'may', 'come', 'and', 'go', 'or', 'you', 'may', 'develop', 'atrial', 'fibrillation', 'that', 'doesn', 't', 'go', 'away', 'and', 'may', 'require', 'treatment', 'although', 'atrial', 'fibrillation', 'itself', 'usually', 'isn', 't', 'life', 'threatening', 'it', 'is', 'a', 'serious', 'medical', 'condition', 'that', 's

it's accuracy is very low because I have only trained it for 5 epochs just to test, we have to train a model for atleast 100 epochs in order to get our desired result. 